In [ ]:
antibody antigen
FV fragmen is fv4155

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
%load_ext autoreload
%autoreload 2 

project_dir = os.path.dirname(os.getcwd())
print('project direcotry is ', project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)
print(sys.path)

project direcotry is  /home/yuan/bio/antibody-antigen-complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/antibody-antigen-complex']


## view structure

In [3]:
# get *pdb
from src.download import Download

pdb_id = '1BFV'
pdb_file = Download.download_pdb(pdb_id)

../pdb/1BFV.pdb is ready.


In [4]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(pdb_file).read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex

In [5]:
# get chains
from src.parse_abag import ParseAbAg

outputs_dir = '../outputs'
p = ParseAbAg(pdb_file, outputs_dir)
# p.structure.header

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 1BFV
outputs dir:  ../outputs/1B/1BFV
[{'chains': [{'AA residues': 201,
              'chain_id': 'L',
              'seq': 'DIELTQSPPSLPVSLGDQVSISCRSSQSLVSNNRRNYLHWYLQKPGQSPKLVIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVAAEDLGLYFCSQSSHVPLTFGSGTKLEIKR'},
             {'AA residues': 217,
              'chain_id': 'H',
              'seq': 'QVQLQESGGGLVNLGGSMTLSCVASGFTFNTYYMSWVRQTPEKTLELVAAINSDGEPIYYPDTLKGRVTISRDNAKKTLYLQMSSLNFEDTALYYCARLNYAVYGMDYWGQGTTVTVSS'}],
  'model_id': 0}]


In [6]:
p.structure.header['compound']

{'1': {'misc': '',
  'molecule': 'fv4155',
  'chain': 'l',
  'fragment': 'monoclonal antibody fv fragment',
  'engineered': 'yes'},
 '2': {'misc': '',
  'molecule': 'fv4155',
  'chain': 'h',
  'fragment': 'monoclonal antibody fv fragment',
  'engineered': 'yes'}}

In [7]:
# filter: is antibody-antigen complex
p.filter_antibody_antigen()
p.update_chains('chain_type')
print(p.structure_id)
pprint(p.info)

Pairwise chains of 1BFV are updated.
1BFV
[{'H': 'single', 'L': 'single'}]


In [8]:
# split chains
p.chains_to_pdb()

Saved chain ['L'] to ../outputs/1B/1BFV/1BFV_L.pdb
Saved chain ['H'] to ../outputs/1B/1BFV/1BFV_H.pdb
Saved chain ['L', 'H'] to ../outputs/1B/1BFV/1BFV_L_H.pdb


In [9]:
p.bounded

[{'type': 'single-single',
  'chains': [['L'], ['H'], ['L', 'H']],
  'pdb_files': ['../outputs/1B/1BFV/1BFV_L.pdb',
   '../outputs/1B/1BFV/1BFV_H.pdb',
   '../outputs/1B/1BFV/1BFV_L_H.pdb']}]

In [10]:
p.save_annot()

https://www.ebi.ac.uk/pdbe/api/mappings/1bfv
Save annotation of 1BFV: ../outputs/1B/1BFV/meta.json


## SASA

In [11]:
# calculate SASA
sasa_file = p.build_freesasa()

In [12]:
from src.analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa(sasa_file)
p2.print_meta()
print(list(p2.data[0]['delta_total_sasa']))

key1 ['type', 'chains', 'pdb_files', 'sasa_files', 'delta_total_sasa']
single chain L ~ single chain H
------------------------------
['delta1', 'delta1_file', 'delta2', 'delta2_file']


In [13]:
# retrieve sequences
motifs = p2.retrieve_seq()
print('output file: ', motifs[1])
motifs

output file:  ../outputs/1B/1BFV/seq_sasa_delta-0_span-30.p


([('single',
   [{'seq': 'D',
     'sig_res': 1,
     'start': (0, '1'),
     'end': (0, '1'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'RRNYLHWYLQKPGQSPKLVIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVAAEDLGLYFCSQSSHVPLTFG',
     'sig_res': 23,
     'start': (33, '34'),
     'end': (103, '104'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}],
   'single',
   [{'seq': 'VRQTPEKTLELVAAINSDGEPIY',
     'sig_res': 8,
     'start': (36, '37'),
     'end': (58, '59'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'YCARLNYAVYGMDYWGQ',
     'sig_res': 14,
     'start': (94, '95'),
     'end': (110, '111'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}])],
 '../outputs/1B/1BFV/seq_sasa_delta-0_span-30.p')

In [14]:
# dataset for ML
seq_sasa_file = motifs[1]
AnalyzeSasa(seq_sasa_file).build_dataset()

[{'input': 'VRQTPEKTLELVAAINSDGEPIY | YCARLNYAVYGMDYWGQ',
  'input_label': 'single',
  'output': 'D | RRNYLHWYLQKPGQSPKLVIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVAAEDLGLYFCSQSSHVPLTFG',
  'output_label': 'single'}]

In [15]:
# complex
p2.print_delta_sasa(0)

complex: single chain L ~ single chain H
------------------------------
single chain:
    chain_id res_no  res aa      value
0          L      1  ASP  D   4.111515
33         L     34  ARG  R  26.271267
36         L     37  TYR  Y  13.803103
38         L     39  HIS  H  18.639044
40         L     41  TYR  Y  53.968616
42         L     43  GLN  Q  42.348619
46         L     47  GLN  Q   2.997984
47         L     48  SER  S  55.652334
48         L     49  PRO  P  72.770746
49         L     50  LYS  K   2.703881
50         L     51  LEU  L  34.388024
53         L     54  TYR  Y  39.879402
54         L     55  LYS  K  27.074195
59         L     60  PHE  F  63.996203
60         L     61  SER  S   8.547092
89         L     90  LEU  L  25.686868
91         L     92  PHE  F  48.938273
95         L     96  SER  S  22.656111
98         L     99  VAL  V  12.167464
99         L    100  PRO  P  40.558212
100        L    101  LEU  L  71.829927
101        L    102  THR  T   1.351940
102        L    1

## distance

In [16]:
# calculate distance
dist_file = p.build_dist()

In [17]:
from src.analyze_dist import AnalyzeDist

max_span = 30
p3= AnalyzeDist(dist_file, max_span=max_span)
p3.print_meta()
print(list(p3.data[0]['dist']))

key1 ['type', 'chains', 'pdb_files', 'dist']
single chain L ~ single chain H
------------------------------
['dist1', 'dist1_file', 'dist2', 'dist2_file']


In [18]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs

([('single',
   [{'seq': 'RRNYLHWYLQKPGQSPKLVIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVAAEDLGLYFCSQSSHVPLTF',
     'sig_res': 21,
     'start': (33, 34),
     'end': (102, 103),
     'dist_cutoff': 5,
     'max_span': 30}],
   'single',
   [{'seq': 'VRQTPEKTLELVAAINSDGEPIY',
     'sig_res': 8,
     'start': (36, 37),
     'end': (58, 59),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'YCARLNYAVYGMDYWGQ',
     'sig_res': 14,
     'start': (94, 95),
     'end': (110, 111),
     'dist_cutoff': 5,
     'max_span': 30}])],
 '../outputs/1B/1BFV/seq_dist_dist-5_span-30.p')

In [19]:
seq_dist_file = motifs[1]
AnalyzeDist(seq_dist_file).build_dataset()

[{'input': 'VRQTPEKTLELVAAINSDGEPIY | YCARLNYAVYGMDYWGQ',
  'input_label': 'single',
  'output': 'RRNYLHWYLQKPGQSPKLVIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVAAEDLGLYFCSQSSHVPLTF',
  'output_label': 'single'}]

In [20]:
# complex
p3.print_min_dist(0)

------------------------------
     res_no  res atom      value aa
0         1  ASP    C   5.168054  D
1         2  ILE    C   8.021149  I
2         3  GLU    C   7.290709  E
3         4  LEU    C   8.717399  L
4         5  THR    C  11.294782  T
..      ...  ...  ...        ... ..
108     109  LEU    C  10.888915  L
109     110  GLU    C  11.091922  E
110     111  ILE    C  15.194666  I
111     112  LYS    C  18.144375  K
112     113  ARG    C  19.755758  R

[113 rows x 5 columns]
------------------------------
     res_no  res atom     value aa
36       37  VAL    C  4.086007  V
38       39  GLN    C  2.743940  Q
42       43  LYS    C  3.180643  K
43       44  THR    C  4.901959  T
44       45  LEU    C  3.608084  L
45       46  GLU    C  4.225690  E
46       47  LEU    C  4.166849  L
58       59  TYR    C  3.397779  Y
94       95  TYR    C  2.849705  Y
98       99  LEU    C  4.967941  L
99      100  ASN    C  4.217044  N
100     101  TYR    C  4.781954  Y
101     102  ALA    C  3.02